# 필작이의 chatbot




In [ ]:
# openai, streamlit, localtunnel 설치
!pip install openai==0.28
!pip install streamlit
!npm install localtunnel
!pip install -U typing_extensions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.254s

3 packages are looking for funding
  run `n

In [ ]:


%%writefile elastic_gpt_app.py


import os
import streamlit as st
import openai
import json
from elasticsearch import Elasticsearch

#openai apy key 입력
openai_api = ''


openai.api_key = openai_api
model = "gpt-3.5-turbo-0301"

def es_connect(cid, user, passwd):
    es = Elasticsearch(cloud_id=cid, http_auth=(user, passwd))
    return es

# Elasticsearch Search
def search(query_text):
    # ES Cloud ID, Username, Password 입력
    cid = ''
    cu = ''
    cp = ''

    es = es_connect(cid, cu, cp)

    # 엘라스틱 검색
    query = {
        "bool": {
          "should": [{
            "match": {
              "EXPLAIN": {
                "query": query_text,
                "boost": 50
              }
            }
          },
          {
          "match_phrase_prefix": {
            "EXPLAIN": {
              "query": query_text,
              "boost": 1
            }
          }
          }
      ],
      "filter": [{
        "exists": {
          "field": "EXPLAIN"
          }
      }],
            "minimum_should_match": 1
        }
    }
    sort={"_score": {"order": "desc"}}
    index = 'buildings_2'
    fields = ["CITY", "EXPLAIN", "ADDRESS", "THEME", "NAME", "IMAGE"]

    resp = es.search(index=index,
                     query=query,
                     fields=fields,
                     size=1, # 상위 3개의 문서 가져옴
                     sort=sort,
                     source=False
                     )
    cities = []
    explains = []
    addresses = []
    themes = []
    names = []
    images = []


    if 'hits' in resp and 'hits' in resp['hits'] and resp['hits']['hits']:
      for hit in resp['hits']['hits']:
          city = hit['fields']['CITY'][0]
          explain = hit['fields']['EXPLAIN'][0]
          address = hit['fields']['ADDRESS'][0]
          theme = hit['fields']['THEME'][0]
          name = hit['fields']['NAME'][0]
          image = hit['fields']['IMAGE'][0]

          cities.append(city)
          explains.append(explain)
          addresses.append(address)
          themes.append(theme)
          names.append(name)
          images.append(image)

    return cities, explains, addresses, themes, names, images




def truncate_text(text, max_tokens):
    tokens = text.split()
    if len(tokens) <= max_tokens:
        return text

    return ' '.join(tokens[:max_tokens])

# Chat_gpt 답변 생성
def chat_gpt(prompt, model="gpt-3.5-turbo", max_tokens=1024, max_context_tokens=4000, safety_margin=5):
    # Truncate the prompt content to fit within the model's context length
    truncated_prompt = truncate_text(prompt, max_context_tokens - max_tokens - safety_margin)

    response = openai.ChatCompletion.create(model=model,
                                            messages=[{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": truncated_prompt}])

    return response["choices"][0]["message"]["content"]

st.title("Building GPT")

# 입력창
with st.form("chat_form"):
    query = st.text_input("You: ") # 사용자에게 질문 받기
    submit_button = st.form_submit_button("Send")


# 답변 출력
negResponse = "해당 질문에 관련된 폐건물정보가 없습니다."
if submit_button:
    gcity, gexplain, gaddress, gtheme, gname, gimage = search(query)
    prompt = f"'{query}'에 대한 답을 다음 정보를 활용해 폐건물을 추천해주세요. 사람처럼 말해주세요.: {gcity}, {gexplain}, {gaddress}, {gtheme}, {gname}\nIf the answer is not contained in the supplied doc reply '{negResponse}' and nothing else"
    answer = chat_gpt(prompt)

    st.write(f"ChatGPT: {answer.strip()}")

    # 이미지 표시

    st.image(gimage, use_column_width=True)




Overwriting elastic_gpt_app.py


In [ ]:


%%writefile elastic_gpt_app.py


import os
import streamlit as st
import openai
import json
from elasticsearch import Elasticsearch
from urllib.parse import quote

#openai apy key 입력
openai_api = ''


openai.api_key = openai_api
model = "gpt-3.5-turbo-0301"

def es_connect(cid, user, passwd):
    es = Elasticsearch(cloud_id=cid, http_auth=(user, passwd))
    return es

# Elasticsearch Search
def search(query_text):
    # ES Cloud ID, Username, Password 입력
    cid = ''
    cu = ''
    cp = ''

    es = es_connect(cid, cu, cp)

    # 엘라스틱 검색
    query = {
        "bool": {
          "should": [{
            "match": {
              "EXPLAIN": {
                "query": query_text,
                "boost": 50
              }
            }
          },
          {
          "match_phrase_prefix": {
            "EXPLAIN": {
              "query": query_text,
              "boost": 1
            }
          }
          }
      ],
      "filter": [{
        "exists": {
          "field": "EXPLAIN"
          }
      }],
            "minimum_should_match": 1
        }
    }
    sort={"_score": {"order": "desc"}}
    index = 'old_ones'
    fields = ["CITY", "EXPLAIN", "ADDRESS", "THEME", "NAME", "IMAGE"]

    resp = es.search(index=index,
                     query=query,
                     fields=fields,
                     size=1, # 상위 3개의 문서 가져옴
                     sort=sort,
                     source=False
                     )
    cities = []
    explains = []
    addresses = []
    themes = []
    names = []
    images = []


    if 'hits' in resp and 'hits' in resp['hits'] and resp['hits']['hits']:
      for hit in resp['hits']['hits']:
          city = hit['fields']['CITY'][0]
          explain = hit['fields']['EXPLAIN'][0]
          address = hit['fields']['ADDRESS'][0]
          theme = hit['fields']['THEME'][0]
          name = hit['fields']['NAME'][0]
          image = hit['fields']['IMAGE'][0]

          cities.append(city)
          explains.append(explain)
          addresses.append(address)
          themes.append(theme)
          names.append(name)
          images.append(image)

    return cities, explains, addresses, themes, names, images




def truncate_text(text, max_tokens):
    tokens = text.split()
    if len(tokens) <= max_tokens:
        return text

    return ' '.join(tokens[:max_tokens])

# Chat_gpt 답변 생성
def chat_gpt(prompt, model="gpt-3.5-turbo", max_tokens=1024, max_context_tokens=4000, safety_margin=5):
    # Truncate the prompt content to fit within the model's context length
    truncated_prompt = truncate_text(prompt, max_context_tokens - max_tokens - safety_margin)

    response = openai.ChatCompletion.create(model=model,
                                            messages=[{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": truncated_prompt}])

    return response["choices"][0]["message"]["content"]




st.title("Building GPT")

# 입력창
with st.form("chat_form"):
    query = st.text_input("You: ") # 사용자에게 질문 받기
    submit_button = st.form_submit_button("Send")


# 답변 출력
negResponse = "해당 질문에 관련된 폐건물정보가 없습니다."
if submit_button:
    gcity, gexplain, gaddress, gtheme, gname, gimage = search(query)
    prompt = f"'{query}'에 대한 답을 다음 정보를 활용해 폐건물을 추천해주세요. 사람처럼 말해주세요.: {gcity}, {gexplain}, {gaddress}, {gtheme}, {gname}\nIf the answer is not contained in the supplied doc reply '{negResponse}' and nothing else"
    answer = chat_gpt(prompt)

    # 이미지 URL
    #escaped_url = quote(gimage, safe=':/')
    #escaped_url = quote(gimage.encode('utf-8'), safe=':/')
    #url = gimage[0]
    #escaped_url = quote(url.encode('utf-8'), safe=':/')

    # 이미지 표시
    with st.container():
      st.image(gimage, use_column_width=True)




Overwriting elastic_gpt_app.py


In [ ]:
!pip install elasticsearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.1/432.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 8.0 MB/s eta 0:00:00


## Web UI(제대로 출력되는지 확인하는 파트)

In [ ]:
# Streamlit 실행
!streamlit run /content/elastic_gpt_app.py &>/content/logs.txt &

```python
🐘
""
UI를 콘솔이 아닌 웹에서 보기 위해 임시 페이지를 열었어요.
아래에서 확인되는 Endpoint IP를 복사하고, npx에서 출력해주는 임시 URL에 들어가 입력해 주세요!
""
```

In [ ]:
# Endpoint IP 출력
import urllib
print("IP Endpoint:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

IP Endpoint: 35.223.195.19


In [ ]:
# 임시 URL 발급
!npx localtunnel --port 8501

npx: installed 22 in 2.093s
your url is: https://open-cars-swim.loca.lt


In [ ]:
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
